# TasteBud: GAN Based Recipe Generation with Graph

Introductory words about this project...

----
## Data Processing

Some words about source and process...

In [33]:
import csv
import numpy as mp
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torchtext
import pandas as pd
import matplotlib.pyplot as plt
import os.path
import ast
import glob

### Data Subsetting
Since RecipeNGL contains 2.23 million recipes, select a subset to use:

In [101]:
full_ngl_path = './data/recipe_ngl/full_dataset.csv'
# if the full recipeNGL dataset csv file exists, read a subset of it
if os.path.exists(full_ngl_path):
    ngl_subset = [0, 100]
    df = pd.read_csv(full_ngl_path, skiprows=ngl_subset[0], nrows=ngl_subset[1], index_col=0)
    df.to_csv(f'./data/recipe_ngl/dataset_{ngl_subset[0]}_{ngl_subset[1]}.csv')

for file in glob.glob('./data/recipe_ngl/dataset*.csv'):
    print(file)
    df = pd.read_csv(file, index_col=0, 
                     converters={'ingredients':pd.eval, 'directions':pd.eval, 'NER':pd.eval})

./data/recipe_ngl\dataset_0_100.csv


In [53]:
df[:3]

,title,ingredients,directions,link,source,NER
0,No-Bake Nut Cookies,"[1 c. firmly packed brown sugar, 1/2 c. evapor...","[In a heavy 2-quart saucepan, mix brown sugar,...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[brown sugar, milk, vanilla, nuts, butter, bit..."
1,Jewell Ball'S Chicken,"[1 small jar chipped beef, cut up, 4 boned chi...","[Place chipped beef on bottom of baking dish.,...",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[beef, chicken breasts, cream of mushroom soup..."
2,Creamy Corn,"[2 (16 oz.) pkg. frozen corn, 1 (8 oz.) pkg. c...","[In a slow cooker, combine all ingredients. Co...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[frozen corn, cream cheese, butter, garlic pow..."


### Create List of Ingredients

In [70]:
ingredients_set = set()
for i in range(len(df["NER"])):
    ingredients_set = ingredients_set | set(df["NER"][i])
print(ingredients_set)

{'dark sweet pitted cherries', 'garlic salt', 'nutmeg', 'orange flavoring', 'red apples', 'Tomato', 'nut meats', 'whole cloves', 'ground white pepper', 'fruit cocktail', 'cocoa', 'apple', 'flavor gelatin', 'Mozzarella cheese', 'kidney beans', 'thin green onion strips', 'lemons', 'garlic', 'Crisco oil', 'cheese', 'orange', 'chicken cutlets', 'green beans', 'condensed milk', 'cream-style', 'bell peppers', 'pepper', 'carrot chunks', 'extra lean ground beef', 'mayonnaise', 'macaroni', 'sweet basil', 'corn tortilla', 'buttermilk', 'corn', 'chocolate fudge pudding', 'broccoli', 'bell pepper', 'chocolate fudge cake', 'green chili peppers', 'Italian tomatoes', 'orange juice', 'mushrooms', 'cheddar cheese', 'pie filling', 'rhubarb', 'shell', 'pear halves', 'jars Junior baby food', 'egg whites', 'cold water', 'oleo', 'lemon yogurt', 'mustard', 'crackers', 'yellow cake mix', 'onion', 'applesauce', 'chicken broth', 'black pepper', 'strawberry jello', 'Italian dressing', 'whipping cream', 'stuffing

In [69]:
cw = csv.writer(open("data/ingredients_list.csv",'w'))
cw.writerow(list(ingredients_set))

3152

### Data Cleaning

In [90]:
# filter for only "Gathered" sources, since those have more consistent format
gathered_df = df[df.source == 'Gathered']
# remove unnecessary columns
filtered_df = gathered_df[['title', 'ingredients', 'directions', 'NER']]

In [91]:
filtered_df[:7]

,title,ingredients,directions,NER
0,No-Bake Nut Cookies,"[1 c. firmly packed brown sugar, 1/2 c. evapor...","[In a heavy 2-quart saucepan, mix brown sugar,...","[brown sugar, milk, vanilla, nuts, butter, bit..."
1,Jewell Ball'S Chicken,"[1 small jar chipped beef, cut up, 4 boned chi...","[Place chipped beef on bottom of baking dish.,...","[beef, chicken breasts, cream of mushroom soup..."
2,Creamy Corn,"[2 (16 oz.) pkg. frozen corn, 1 (8 oz.) pkg. c...","[In a slow cooker, combine all ingredients. Co...","[frozen corn, cream cheese, butter, garlic pow..."
3,Chicken Funny,"[1 large whole chicken, 2 (10 1/2 oz.) cans ch...","[Boil and debone chicken., Put bite size piece...","[chicken, chicken gravy, cream of mushroom sou..."
4,Reeses Cups(Candy),"[1 c. peanut butter, 3/4 c. graham cracker cru...",[Combine first four ingredients and press in 1...,"[peanut butter, graham cracker crumbs, butter,..."
5,Cheeseburger Potato Soup,"[6 baking potatoes, 1 lb. of extra lean ground...",[Wash potatoes; prick several times with a for...,"[baking potatoes, extra lean ground beef, butt..."
6,Rhubarb Coffee Cake,"[1 1/2 c. sugar, 1/2 c. butter, 1 egg, 1 c. bu...","[Cream sugar and butter., Add egg and beat wel...","[sugar, butter, egg, buttermilk, flour, salt, ..."


### Data Tokenizing

In [124]:
tokenized_titles = []
tokenized_ingredients = []
tokenized_directions = []
tokenized_NER = []

for i, row in filtered_df.iterrows():
    # tokenize titles
    tokens_list = filtered_df.title.values[i].split(' ')
    tokenized_titles.append(tokens_list)
    
    # tokenize ingredients
    tokens_list = []
    for ingredient_item in row.ingredients:
        tokens_list.append(ingredient_item.split(' '))
    tokenized_ingredients.append(tokens_list)
    
    # tokenize directions
    tokens_list = []
    for direction_item in row.directions:
        tokens_list.append(direction_item.split(' '))
    tokenized_directions.append(tokens_list)
    
    # tokenize ingredients
    tokens_list = []
    for NER_item in row.NER:
        tokens_list.append(NER_item.split(' '))
    tokenized_NER.append(tokens_list)

print(tokenized_titles[0])
print(tokenized_ingredients[0])
print(tokenized_directions[0])
print(tokenized_NER[0])

['No-Bake', 'Nut', 'Cookies']
[['1', 'c.', 'firmly', 'packed', 'brown', 'sugar'], ['1/2', 'c.', 'evaporated', 'milk'], ['1/2', 'tsp.', 'vanilla'], ['1/2', 'c.', 'broken', 'nuts', '(pecans)'], ['2', 'Tbsp.', 'butter', 'or', 'margarine'], ['3', '1/2', 'c.', 'bite', 'size', 'shredded', 'rice', 'biscuits']]
[['In', 'a', 'heavy', '2-quart', 'saucepan,', 'mix', 'brown', 'sugar,', 'nuts,', 'evaporated', 'milk', 'and', 'butter', 'or', 'margarine.'], ['Stir', 'over', 'medium', 'heat', 'until', 'mixture', 'bubbles', 'all', 'over', 'top.'], ['Boil', 'and', 'stir', '5', 'minutes', 'more.', 'Take', 'off', 'heat.'], ['Stir', 'in', 'vanilla', 'and', 'cereal;', 'mix', 'well.'], ['Using', '2', 'teaspoons,', 'drop', 'and', 'shape', 'into', '30', 'clusters', 'on', 'wax', 'paper.'], ['Let', 'stand', 'until', 'firm,', 'about', '30', 'minutes.']]
[['brown', 'sugar'], ['milk'], ['vanilla'], ['nuts'], ['butter'], ['bite', 'size', 'shredded', 'rice', 'biscuits']]


In [125]:
tokenized_df = filtered_df.copy()
tokenized_df['token_title'] = tokenized_titles
tokenized_df['token_ingredients'] = tokenized_ingredients
tokenized_df['token_directions'] = tokenized_directions
tokenized_df['token_NER'] = tokenized_NER

In [126]:
tokenized_df[:7]

,title,ingredients,directions,NER,token_title,token_ingredients,token_directions,token_NER
0,No-Bake Nut Cookies,"[1 c. firmly packed brown sugar, 1/2 c. evapor...","[In a heavy 2-quart saucepan, mix brown sugar,...","[brown sugar, milk, vanilla, nuts, butter, bit...","[No-Bake, Nut, Cookies]","[[1, c., firmly, packed, brown, sugar], [1/2, ...","[[In, a, heavy, 2-quart, saucepan,, mix, brown...","[[brown, sugar], [milk], [vanilla], [nuts], [b..."
1,Jewell Ball'S Chicken,"[1 small jar chipped beef, cut up, 4 boned chi...","[Place chipped beef on bottom of baking dish.,...","[beef, chicken breasts, cream of mushroom soup...","[Jewell, Ball'S, Chicken]","[[1, small, jar, chipped, beef,, cut, up], [4,...","[[Place, chipped, beef, on, bottom, of, baking...","[[beef], [chicken, breasts], [cream, of, mushr..."
2,Creamy Corn,"[2 (16 oz.) pkg. frozen corn, 1 (8 oz.) pkg. c...","[In a slow cooker, combine all ingredients. Co...","[frozen corn, cream cheese, butter, garlic pow...","[Creamy, Corn]","[[2, (16, oz.), pkg., frozen, corn], [1, (8, o...","[[In, a, slow, cooker,, combine, all, ingredie...","[[frozen, corn], [cream, cheese], [butter], [g..."
3,Chicken Funny,"[1 large whole chicken, 2 (10 1/2 oz.) cans ch...","[Boil and debone chicken., Put bite size piece...","[chicken, chicken gravy, cream of mushroom sou...","[Chicken, Funny]","[[1, large, whole, chicken], [2, (10, 1/2, oz....","[[Boil, and, debone, chicken.], [Put, bite, si...","[[chicken], [chicken, gravy], [cream, of, mush..."
4,Reeses Cups(Candy),"[1 c. peanut butter, 3/4 c. graham cracker cru...",[Combine first four ingredients and press in 1...,"[peanut butter, graham cracker crumbs, butter,...","[Reeses, Cups(Candy), , ]","[[1, c., peanut, butter], [3/4, c., graham, cr...","[[Combine, first, four, ingredients, and, pres...","[[peanut, butter], [graham, cracker, crumbs], ..."
5,Cheeseburger Potato Soup,"[6 baking potatoes, 1 lb. of extra lean ground...",[Wash potatoes; prick several times with a for...,"[baking potatoes, extra lean ground beef, butt...","[Cheeseburger, Potato, Soup]","[[6, baking, potatoes], [1, lb., of, extra, le...","[[Wash, potatoes;, prick, several, times, with...","[[baking, potatoes], [extra, lean, ground, bee..."
6,Rhubarb Coffee Cake,"[1 1/2 c. sugar, 1/2 c. butter, 1 egg, 1 c. bu...","[Cream sugar and butter., Add egg and beat wel...","[sugar, butter, egg, buttermilk, flour, salt, ...","[Rhubarb, Coffee, Cake]","[[1, 1/2, c., sugar], [1/2, c., butter], [1, e...","[[Cream, sugar, and, butter.], [Add, egg, and,...","[[sugar], [butter], [egg], [buttermilk], [flou..."


### Ingredient Graph

----
## Primary Model

### Ingredient Selector

### GAN RNN

### Training

----
## Baseline Model

----
## Results and Comparison